In [1]:
!pip -q install flask-ngrok
!mkdir -p /root/.ngrok2/
!echo "authtoken: ###" > /root/.ngrok2/ngrok.yml

In [ ]:
!pip install bitsandbytes accelerate xformers einops langchain faiss-cpu transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever


device = 'cuda' if torch.cuda.is_available() else 'cpu'
from flask import jsonify, request
from flask import Flask
from flask_ngrok import run_with_ngrok


orig_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
                               )
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(orig_model_path)
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=100,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    model_kwargs={"device": "cuda"},
)

db_docs = [
    "You are a assistant about LG Electronic Air conditioner",
    "Low cooling issue comes in air conditioner mainly due to following reasons :\n1. Unset Cooling mode\n2. Dirty and Blocked filters\n3. Low Voltage as per requirement (200V-240V)\nTo resolve the no cooling issue, please follow the steps listed below\nStep 1: Check the mode setting and select the cooling mode.\nStep 2: Select the temperature setting and set the temperature on 18°C\nStep 3: Select the fan speed at ‘High’ mode.\nStep 4: Clean the air filter if it is dusty.\nStep 5: Check the voltage. It should be in between 200V-240V.",
    "AC Filter Cleaning helps in preventive maintenance of the air conditioner and keeps the AC clean, odor free, improves energy efficiency & gives better cooling performance.\nTo clean the AC filter, please follow steps listed below :\nStep 1: Hold the knobs of the air filter and lift it up slightly and take out the air filter from AC indoor unit.\nStep 2: Clean the air filter with normal temperature water and brush.\nStep 3: Wait until the air filter completely become dry.\nStep 4: Re-fix the dry and cleansed air filter in the AC indoor unit.\n*Note: - For better cooling performance, clean the AC filter in every 15 days.\n*You can also check below link for video tutorial\nhttps://www.lg.com/in/support/video-tutorials/CT20150054-20151164081818",
    "No power issue comes in air conditioner mainly due to following reasons : \n 1. AC is not powered on from remote\n2. MCB board may be turned 'OFF'\nTo resolve the no power issue, please follow the steps listed below :\nStep 1: Check the MCB condition, It must be 'ON'.\nStep 2: Check voltage. It should be in between 200V-240V..",
    "Water drip out through air conditioner due to bent or twisted drain pipes To resolve the leaking issue, please follow steps listed below :\nStep 1: Drain pipe should be inclined downward direction.\nStep 2: End point of drain pipe should not be dipped into water body/bucket\nStep 3: Clean the air filter after every 20 days.\nStep 4: Run the AC on M'SOON Mode",
    "How To Get Rid Of Water Drops/Moisture Problem?During the rainy season, moisture comes on the air conditioner due to high humidity in air.\nTo resolve the water droplets on AC panel issue, please follow steps listed below :\nStep 1: This is a process of circulating cold refrigerant in pipe.\nStep 2: Outdoor Unit always be kept under shade.\nStep 3: Press 'Mode' key until ‘Drop’ symbol is reflecting on Remote.\n*Note: - It is normal to have light water drops due to excessive moisture in rainy season.",
    "How To Solve Odor Problem?When the AC filter is not cleansed for a long time then water start collecting on the indoor unit which causes bad smell.\nTo reduce the odor/bad smell, please follow steps listed below :\nClean Air Filter :\n• Step 1 : Hold the knobs of the air filter and lift it up slightly and take out the air filter from AC indoor unit.\n• Step 2 : Clean the air filter with normal temperature water and brush.\n• Step 3 : Wait until the air filter completely become dry.\n• Step 4 : Re-fix the dry and cleansed air filter in the AC indoor unitAuto Clean Mode :\n• Step 1 : Press power button to turn on the AC.\n• Step 2 : Press A/Clean Button\n• Step 3 : Press power button again to turn off the AC.\n• Step 4 : Clean the air filter in every 15 days.\n*Note: - 'A/Clean' runs 30 minutes. During this mode, keep the doors and windows open.",
    "How To Fix Noise Issue?Q 1. 'Hissing' sound is coming from Air Conditioner\nAns. Hissing sound is caused by the refrigerant flows inside the coils of AC. It is a normal phenomenon of a fine working AC.\nQ 2. 'Humming' sound is coming from Air Conditioner.\nAns. Humming sound comes when compressor runs. It is a normal phenomenon of a fine working AC.\nQ 3. Noise is coming from front grill of Air Conditioner\nAns. The Sound from front grill of AC is caused by the refrigerant flows inside the coils of AC. It is a normal phenomenon of a fine working AC. Check the grill fitment. Gently push at its two ends.\nQ 4. 'Water Flowing' sound is coming from Air Conditioner.\nAns. The Water Flowing sound is caused by the refrigerant flows inside the AC. It is a normal phenomenon of a fine working AC.\nQ 5. 'Clicking' sound is coming from Air Conditioner.\nAns. The Clicking sound occurs when the air filter is not fitted correctly so remove the air filter and re-fix it correctly. To reduce the clicking sound, please follow steps listed below :\nStep 1 : Adjust the front grill.\nStep 2 : Adjust the air filter by opening front grill.\n*Note: - Use stabilizer in case of too low or too high voltage.\nFor 1 Ton AC 3 kVA stabilizer requires\nFor 1.5 Ton AC 4 kVA stabilizer requires\nFor 20. Ton AC 5 kVA stabilizer requires\n• If the issue still persist, please contact us on our WhatsApp 9711709999",
    "With an air conditioner, you may not find a one-size-fits-all solution. The AC that efficiently cools your bedroom might not be the best choice for your living room. Thus, you must understand your needs and requirements to make the best decision. Also, you should identify which air conditioning solution, split or window AC, is more suitable for cooling your house. While split ACs are more efficient but difficult to install, window ACs are simpler to set up but can only cool one or two rooms.",
    "A few things you should consider before investing in an air conditioner include:\nRoom size\nCooling capacity\nEnergy-efficiency\nType of AC\nFeatures\nTonnage of AC\nStar Rating\nPrice",
    "Follow these steps to ensure proper air conditioner maintenance:\n● Since the air filters collect a large amount of dust and dirt over time, you should clean them regularly.\n● Check for leakage and see if the water is dripping from the system.\n● The dirt and dust can accumulate over the condenser and evaporator coils. So, you must clean it.\n● The evaporator drain gets clogged due to algae or mould build-up. You should unplug and clean it to ensure proper functioning.\n● The condenser fan and blower need to rotate freely. So, lubricate them regularly.\n● Since loose connections and wirings can cause equipment failure, you must inspect the wires and controls as part of AC maintenance.",
    "The air conditioners are given a star rating by the BEE (Bureau of Energy Efficiency), which defines their efficiency. The more the number of stars, the better its energy efficiency. Using 5-star air conditioning systems helps achieve lower electricity bills, increased energy efficiency, and reduced environmental impact.5-star rated ACs often have advanced features such as better filtration systems and improved airflow. They also have a better life expectancy compared to lower-rated ACs. You should even consider that post-pandemic, many people are working from home. AC usage extends usual hours in this case. Hence using a 5-star rated air conditioner is beneficial for energy efficiency. Additionally, you can recover the cost of a 5-star AC within 1-2 years.",
    "Ideally, you should ensure air conditioner maintenance at least once a year. This will help optimize its performance.",
    "Air conditioners feature three essential components, including an evaporator, condenser, and compressor. They work in tandem with each other to create a cooling effect. Opposite to what many believe, an air conditioner doesn’t produce cool air. Instead, it strips the air of heat to lower its temperature and humidity. This air is then circulated throughout the building to ensure better cooling.",
    "An air conditioner requires periodic maintenance to function well. Over time, the system can get clogged with dirt, bacteria, and dust and cause various health issues like asthma or allergies. Conducting maintenance can help get rid of allergens so you can live healthily. Another reason air conditioners require periodic servicing is to expand their lifespan. An AC in top-notch condition runs long and experiences fewer breakdowns. Moreover, well-maintained systems ensure efficient cooling and incur fewer running costs.",
    "An inverter AC can operate at different speeds to ensure the room is cooled at an optimum temperature. It can adjust the temperature without regularly turning the motor on/off. Here are a few advantages of opting for this system:\n● Inverter AC consume less power than the traditional systems\n● They are environmental-friendly\n● These systems ensure effective cooling\n● They produce less noise compared with other types of ACs\n● Inverter ACs help maintain a constant temperature\n● They are safe for residential wiring",
    "While conserving energy is a good habit, turning the system off every time you leave the room can actually lead to energy wastage. Although it may not harm the system, it can increase expenses. So, the best solution is to crank up the temperature when you leave the room. This will require your air conditioning unit to work less and lead to reduced energy bills.",
    "Here are a few tips you must practise to reduce your energy bills:\n● Running the AC at a minimum temperature doesn’t ensure better cooling. So, set the temperature at 24 degrees or higher to save energy and cut the electricity bills.\n● When the AC is not in use, turn off the power switch. While most people prefer turning off the system using a remote, it is not ideal as electricity is wasted when the compressor is set to idle load.\n● Similar to other electrical appliances, air conditioning systems also need servicing. Thus, you should contact HVAC maintenance to look at your AC at least twice a year.\n● To prevent overusing the AC, use the timer. Set it to turn off the cooling after four or five hours so you can sleep comfortably without compromising your energy bills.",
    "LG air conditioners will feature the model/serial tag mostly on the outside of the unit on the right-hand side as you face the product. It has been noted on some models that for some odd reason, the Model/Serial tag was located on the left-hand side",
		"Operation Indication Lamps\nOn/Off : Lights up during the system operation, error code and auto restart setting.\nSleep mode Energy-Saving Cooling Mode : Lights up during sleep mode auto operation.(non-Inverter Model)\nEnergy-Saving Cooling Mode:Lights up during Energy-Saving Cooling Mode Operation(Inverter Model only)\nPlasma : The function illustrates its dynamic mode by using a multiple lighting system.\nTimer : Lights up during timer operation.(non-Inverter Model)  Lights up during timer operation or  sleep mode auto operation (Inverter Model only)\nDefrost mode: Lights up during defrost mode or *hot start operation (Heat pump model only)\nOutdoor unit operation :Lights up during outdoor unit operation. (Cooling model only)\nPower Display : Indicates the setting temperature and operation mode.",
	  "LG Tower-Type Air Conditioners are cutting-edge cooling solutions designed for both residential and commercial applications. Distinguished by their vertical, tower-like structure, these air conditioners boast a sleek and space-saving design. Here are key features and attributes of LG Tower-Type Air Conditioners:\n1. Space-Efficient Design: The tower-type configuration is specifically engineered to optimize space utilization while delivering robust cooling performance. This design is particularly advantageous for rooms with limited floor space.\n2. Cooling Performance: LG Tower-Type Air Conditioners incorporate advanced cooling technologies, providing efficient and effective cooling for the surrounding environment. They typically offer high cooling capacities and customizable settings to cater to user preferences.\n3. Versatility: These air conditioners are versatile and suitable for diverse settings, including bedrooms, living rooms, offices, or other indoor spaces. Available in different models with varying capacities, they can accommodate various room sizes.\n4. Smart Features: Many LG Tower-Type Air Conditioners are equipped with smart features such as Wi-Fi connectivity and compatibility with smart home systems. This enables users to control the air conditioner remotely through a mobile app or integrate it into their home automation setup.\n5. Energy Efficiency: LG prioritizes energy efficiency in its appliances, and Tower-Type Air Conditioners often feature energy-saving modes and technologies to optimize power consumption, promoting environmental sustainability.\n6. Air Quality Enhancement: Some models come with built-in air purifiers and filters to improve indoor air quality. These features aid in removing dust, allergens, and pollutants, contributing to a healthier indoor environment.\n7. User-Friendly Controls: The control interface is designed to be intuitive and user-friendly, with either physical buttons or touch controls. Certain models may include a digital display for easy monitoring of temperature and settings.\n8. Quiet Operation: LG Tower-Type Air Conditioners are engineered to operate quietly, minimizing noise disruptions in the environment. This is particularly beneficial for bedrooms, offices, or any space where a tranquil atmosphere is desired.\nIn summary, LG Tower-Type Air Conditioners combine a sophisticated design, powerful cooling capabilities, and smart features to deliver an efficient and user-friendly cooling solution for various indoor spaces.",
    "LG Dual Inverter Air Conditioners represent a technologically advanced and energy-efficient cooling solution designed to enhance comfort in residential and commercial spaces. Here's an explanation of the key features of LG Dual Inverter Air Conditioners:\nDual Inverter Technology: The hallmark of these air conditioners is the Dual Inverter compressor, which employs twin rotary motors for more precise and efficient control of the cooling process. This innovative technology helps maintain a consistent temperature while reducing energy consumption.\nHigh Energy Efficiency: LG Dual Inverter Air Conditioners are renowned for their energy efficiency, leading to lower electricity bills and reduced environmental impact. The dual rotary compressor adjusts the cooling capacity to match the precise requirements of the space, optimizing energy usage.\nFast Cooling Performance: With the ability to rapidly cool the room, these air conditioners are equipped to provide quick relief from high temperatures. The Dual Inverter compressor accelerates the cooling process, ensuring a comfortable environment in a short period.\nQuiet Operation: LG prioritizes a quiet and peaceful user experience. Dual Inverter Air Conditioners are engineered to operate at low noise levels, minimizing disruptions in indoor environments such as bedrooms, living rooms, or offices.\nStabilizer-Free Operation: Certain models of LG Dual Inverter Air Conditioners are designed to operate efficiently even in regions with voltage fluctuations, eliminating the need for an external stabilizer.\nSmart Features: Many LG Dual Inverter Air Conditioners come equipped with smart features, such as Wi-Fi connectivity and compatibility with smart home systems. This allows users to control and monitor the air conditioner remotely using a mobile app or integrate it into their home automation setup.\nAdvanced Air Purification: Some models may include features like advanced air filters and purifiers to enhance indoor air quality, capturing and eliminating dust, allergens, and other pollutants.\nUser-Friendly Controls: The control interface is designed to be user-friendly, offering convenient options for adjusting settings and monitoring the temperature. Digital displays and remote control functionality contribute to a seamless user experience.\nIn summary, LG Dual Inverter Air Conditioners combine dual inverter technology, energy efficiency, rapid cooling, and smart features to provide a comprehensive and effective solution for cooling indoor spaces while prioritizing user comfort and convenience.",
    "LG Winner Air Conditioner:\nThe LG Winner Air Conditioner is likely to incorporate innovative features and technologies designed to enhance comfort and energy efficiency in indoor spaces. Some common features found in LG air conditioners include:\nInverter Technology: LG often utilizes inverter technology, which allows for more precise control of the compressor speed, resulting in energy savings and consistent temperature control.\nEnergy Efficiency: LG air conditioners are designed with a focus on energy efficiency, helping users reduce electricity consumption while maintaining optimal cooling performance.\nSmart Functionality: Certain models may offer smart features, enabling users to control and monitor the air conditioner remotely through a dedicated mobile app. This adds convenience and flexibility to the user experience.\nQuiet Operation: LG places importance on minimizing noise levels during operation, ensuring a quiet and comfortable indoor environment.\nAir Purification: Advanced air filtration systems, including filters and purifiers, are often integrated into LG air conditioners to improve indoor air quality by removing particles, allergens, and pollutants.\nUser-Friendly Controls: LG air conditioners typically feature user-friendly control interfaces, with digital displays, remote controls, and easy-to-use buttons for convenient operation.\nFor detailed information about the specific LG Winner Air Conditioner model, including its specifications, features, and user manual, it is recommended to refer to the latest product documentation provided by LG or contact LG customer support for the most accurate and up-to-date information.",
		"LG Khan Air Conditioner: Khan Air Conditioner is likely to incorporate innovative features and technologies designed to enhance comfort and energy efficiency in indoor spaces. Some common features found in LG air conditioners include:\nInverter Technology: LG often utilizes inverter technology, which allows for more precise control of the compressor speed, resulting in energy savings and consistent temperature control.\nEnergy Efficiency: LG air conditioners are designed with a focus on energy efficiency, helping users reduce electricity consumption while maintaining optimal cooling performance.\nSmart Functionality: Certain models may offer smart features, enabling users to control and monitor the air conditioner remotely through a dedicated mobile app. This adds convenience and flexibility to the user experience.\nQuiet Operation: LG places importance on minimizing noise levels during operation, ensuring a quiet and comfortable indoor environment.\nAir Purification: Advanced air filtration systems, including filters and purifiers, are often integrated into LG air"
		"Indoor Unit Faults \n2	:Indoor unit inlet pipe sensor error\n4	:Float switch error (optional)\n5	:Communication error between indoor unit and outdoor units\n6	:Indoor unit outlet pipe sensor error\n9	:Indoor unit EPROM error\n10	:Indoor unit BLDC motor fan lock\n12	:Indoor unit middle pipe sensor error",
		"Outdoor Unit Faults\n22	Current Transformed (CT2) error; Alternating current (AC) input too high\n23	DC Link Low Volt\n25	AC Low/High Volt\n26	DC Comp Position Error (not providing rotation), Locking\n27	PSC Fault; Current to inverter compressor between AC and DC converter circuit too high\n28	Inverter compressor DC voltage is too high\n29	Inverter compressor amperage is too high\n31	Current-to-current transformer (CT) thermistor is too low\n32	Inverter Compressor Discharge Pipe (D-Pipe) Overheat\n40	CT Sensor Error; Thermistor is disconnected or shorted out\n41	D-Pipe Sensor INV is disconnected or shorted out\n44	Outdoor Air Sensor is disconnected or shorted out\n45	Middle thermistor of outdoor unit condenser coil is disconnected or shorted out\n46	Outdoor unit suction line thermistor is disconnected or shorted out\n48	Outdoor unit coil outlet (liquid line) thermistor is disconnected or shorted out\n53	Communication failure from outdoor unit to indoor unit\n60	Outdoor unit printed circuit board (PCB) EPROM check sum error\n61	Outdoor unit condenser coil temperature is too high\n62	Outdoor unit inverter compressor PCB heat sink temperature is too high\n63	Condenser coil pipe thermistor temperature is too low\n65	Heat sink thermistor has disconnected or has shorted out\n67	Outdoor brushless direct current (BLDC) fan motor lock error"
]

vector_db = FAISS.from_texts(db_docs, embeddings)
retriever = VectorStoreRetriever(vectorstore=vector_db)

template ="You are a helpful AI assistant. Use the following pieces of context to answer the question at the end.\
              {context}\
              If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\
              Chat history: {history}\
              Question: {question}\
              Write your answers short. Helpful Answer:"

prompt = PromptTemplate(
        template=template, input_variables=["history", "context", "question"]
    )
qa = RetrievalQA.from_chain_type(
        llm=mistral_llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
            "verbose": False,
            "prompt": prompt,
            "memory": ConversationBufferMemory(
                memory_key="history",
                input_key="question"),
        }
    )

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/chat', methods=["POST", "GET"])
def home():
    input_data = request.get_json()


    input_text = str(input_data.get('input', ''))

    # Ensure input_text is not empty before passing it to qa.run
    if not input_text:
        return jsonify({"error": "Input text is empty"})

    # Assuming qa.run is a function that takes a string input and returns some output
    output = qa.run(input_text)


    '''
    # Send a POST request to another server with the predicted class
    url = 'http://33a9-34-141-212-81.ngrok-free.app/chat'
    data = {"input": return_val}
    headers = {'Content-Type': 'application/json'}

    # Send a POST request to the second server
    r=requests.post(url, json=data, headers=headers)
    '''

    return jsonify({"output": output})

app.run()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://3717-34-147-59-30.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
INFO:werkzeug:127.0.0.1 - - [01/Feb/2024 10:57:35] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Feb/2024 11:01:10] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Feb/2024 11:02:11] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Feb/2024 11:02:30] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Feb/2024 11:09:05] "POST /chat HTTP/1.1" 200 -
